In [ ]:
! pip install -U transformers datasets evaluate seqeval

In [ ]:
!pip install accelerate -U

In [ ]:
! pip install transformers[torch]

In [ ]:
import datasets

In [ ]:
from datasets import Dataset

In [ ]:
ds = Dataset.from_json('data.json')

In [ ]:
labelList = []
for i in range(len(ds)):
  labelList.append(ds[i]['annotations'][0]['label'])

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

dataArray = np.array(ds)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(ds, labelList):
    strat_train_set = list(dataArray[train_index])
    strat_test_set = list(dataArray[test_index])

In [ ]:
nerdataset = datasets.DatasetDict({'train': strat_train_set}) #switch strat_train_set and strat_test_set for following steps to get train set and test set

In [ ]:
label_listner = ['0','DOMAIN_NAME','GEO_LOCATION','URL','MALWARE_NAME','HASH_VAL','ACTOR','FILE_PATH','IP','REG_KEY','EMAIL_ADDRESSES']

In [ ]:
label_listnerI = ['0','DOMAIN_NAME','GEO_LOCATION','URL','MALWARE_NAME','HASH_VAL','ACTOR','FILE_PATH','IP','REG_KEY','EMAIL_ADDRESSES']

In [ ]:
import re

In [ ]:
procesData = []
bull = ['a','an','the','and','on','in','at','of','I']
for i in range(len(nerdataset["train"])):
  texttokens = re.findall(r"[\w']+|[.,!?;]", nerdataset["train"][i]['text'])
  dic = {"text": texttokens,"annotations":[0] * len(texttokens)}

  entityL = []
  for y in range(len(nerdataset["train"][i]['annotations'])):
    entityL.append(nerdataset["train"][i]['annotations'][y]['entity'])

  for f in range(len(texttokens)):
    if f == 0:#在句首用B
      for k in range(len(entityL)):
        if texttokens[f] in entityL[k]:
          if texttokens[f] not in bull:
            index = label_listner.index(nerdataset["train"][i]['annotations'][k]['label'])
            dic['annotations'][f] = index
    else:
      if dic['annotations'][f-1] != 0:#上一个tag不等于0用I
        for k in range(len(entityL)):
          if texttokens[f] in entityL[k]:
            if texttokens[f] not in bull:
              index = label_listnerI.index(nerdataset["train"][i]['annotations'][k]['label'])
              index = index + len(label_listnerI) - 1
              dic['annotations'][f] = index
      else:#上一个tag等于0用B
        for k in range(len(entityL)):
          if texttokens[f] in entityL[k]:
            if texttokens[f] not in bull:
              index = label_listner.index(nerdataset["train"][i]['annotations'][k]['label'])
              dic['annotations'][f] = index

  procesData.append(dic)

In [ ]:
import json

In [ ]:
json.dumps(procesData)

In [ ]:
file = open('nerdata.json', 'w')
file.write(json.dumps(procesData))
file.close()

In [ ]:
pds = Dataset.from_json('nerdata.json')

In [ ]:
pdst = Dataset.from_json('nerdataTest.json')

In [ ]:
pnerdataset = datasets.DatasetDict({'train': pds})

In [ ]:
pnerdatasetTest = datasets.DatasetDict({'train': pdst})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"annotations"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_pnerdatasetTrain = pnerdataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
tokenized_pnerdatasetTest = pnerdatasetTest.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
label_listnerBI = ["O","B-DOMAIN_NAME","B-GEO_LOCATION","B-URL","B-MALWARE_NAME","B-HASH_VAL","B-ACTOR","B-FILE_PATH","B-IP","B-REG_KEY","B-EMAIL_ADDRESSES","I-DOMAIN_NAME","I-GEO_LOCATION","I-URL","I-MALWARE_NAME","I-HASH_VAL","I-ACTOR","I-FILE_PATH","I-IP","I-REG_KEY","I-EMAIL_ADDRESSES"]

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_listnerBI[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_listnerBI[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2labelner = {
    0: "O",
    1: "B-DOMAIN_NAME",
    2: "B-GEO_LOCATION",
    3: "B-URL",
    4: "B-MALWARE_NAME",
    5: "B-HASH_VAL",
    6: "B-ACTOR",
    7: "B-FILE_PATH",
    8: "B-IP",
    9: "B-REG_KEY",
    10: "B-EMAIL_ADDRESSES",
    11: "I-DOMAIN_NAME",
    12: "I-GEO_LOCATION",
    13: "I-URL",
    14: "I-MALWARE_NAME",
    15: "I-HASH_VAL",
    16: "I-ACTOR",
    17: "I-FILE_PATH",
    18: "I-IP",
    19: "I-REG_KEY",
    20: "I-EMAIL_ADDRESSES",
}
label2idner = {
    "O": 0,
    "B-DOMAIN_NAME": 1,
    "B-GEO_LOCATION": 2,
    "B-URL": 3,
    "B-MALWARE_NAME": 4,
    "B-HASH_VAL": 5,
    "B-ACTOR": 6,
    "B-FILE_PATH": 7,
    "B-IP": 8,
    "B-REG_KEY": 9,
    "B-EMAIL_ADDRESSES": 10,
    "I-DOMAIN_NAME": 11,
    "I-GEO_LOCATION": 12,
    "I-URL": 13,
    "I-MALWARE_NAME": 14,
    "I-HASH_VAL": 15,
    "I-ACTOR": 16,
    "I-FILE_PATH": 17,
    "I-IP": 18,
    "I-REG_KEY": 19,
    "I-EMAIL_ADDRESSES": 20,

}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=21, id2label=id2labelner, label2id=label2idner
)

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/trainModel",
    learning_rate=0.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_pnerdatasetTrain["train"],
    eval_dataset=tokenized_pnerdatasetTest["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
metric = load_metric("seqeval")

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_pnerdatasetTest["train"])
predictions1 = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_listnerBI[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions1, labels)
]
true_labels = [
    [label_listnerBI[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions1, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results